In [4]:
%pip install torch==2.3.0 numpy==1.26.4 pandas==2.2.2 scikit-learn==1.4.2


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import numpy as np, pandas as pd, sklearn

print("torch:", torch.__version__)
print("numpy:", np.__version__)
print("pandas:", pd.__version__)
print("scikit-learn:", sklearn.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

torch: 2.3.0+cu121
numpy: 1.26.4
pandas: 2.2.2
scikit-learn: 1.4.2
Sử dụng thiết bị: cuda


In [6]:
train_df = pd.read_csv("../airflow/projects/absa_streaming/data/train_data.csv", sep=",", header=0, encoding='utf-8')
test_df = pd.read_csv("../airflow/projects/absa_streaming/data/test_data.csv", sep=",", header=0, encoding='utf-8')
val_df = pd.read_csv("../airflow/projects/absa_streaming/data/val_data.csv", sep=",", header=0, encoding='utf-8')

print("Shape train_df: ", train_df.shape)
print("Shape test_df: ", test_df.shape)
print("Shape val_df: ",val_df.shape)

COMMENT_COLUMN_NAME = 'Review'
ASPECT_COLUMNS = ['Price', 'Shipping', 'Outlook', 'Quality', 'Size', 'Shop_Service', 'General', 'Others']
# 0: Negative
# 1: Positive
# 2: Neutral
# -1: None

Shape train_df:  (8424, 9)
Shape test_df:  (2340, 9)
Shape val_df:  (936, 9)


In [7]:
import re
import unicodedata

REMOVE_TONE = False
REMOVE_NOISE = True

def remove_vietnamese_tone(text):
    text = unicodedata.normalize('NFD', text)
    text = re.sub(r'[\u0300-\u036f]', '', text)
    text = unicodedata.normalize('NFC', text)
    return text

def remove_non_vietnamese_chars(text):
    return re.sub(
        r"[^a-zàáạảãăắằặẳẵâầấậẩẫèéẹẻẽêềếệểễ"
        r"ìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữ"
        r"ỳýỵỷỹđ0-9\s]",
        " ",
        text
    )

def clean(text):
    text = str(text).lower().strip()

    if REMOVE_TONE:
        text = remove_vietnamese_tone(text)

    if REMOVE_NOISE:
        text = remove_non_vietnamese_chars(text)

    text = re.sub(r"\s+", " ", text)
    return text

train_df["clean_text"] = train_df["Review"].apply(clean).apply(lambda x: " ".join(x.split()))
val_df["clean_text"]   = val_df["Review"].apply(clean).apply(lambda x: " ".join(x.split()))
test_df["clean_text"]  = test_df["Review"].apply(clean).apply(lambda x: " ".join(x.split()))

print(test_df["clean_text"].head(10))




0    giày hơi có mùi nồng lưu ý đôi la không phải đ...
1    hàng về đẹp lắm nha ship thân thiện đi giày vừ...
2                          hàng ôk nên mua dày rất đẹp
3    bun gti gửi oke sớ ơ đi sidbd bởi đi được đạn ...
4    màu đẹp giống trong hình mọi người nên mua nha...
5    chất lượng phù hợp với giá tiền đi đúng sz như...
6    giày trượt lắm huhu đánh giải trường mà trượt ...
7    tr ơi dép đẹp vs dth lắm nha vs giá này mà chấ...
8                                   cũng tạm được thoi
9                   shop hỗ trợ rất tốt mn nên mua nhé
Name: clean_text, dtype: object


In [8]:
from collections import Counter
import json

# Tạo Counter để đếm tần suất từ
counter = Counter()

# Duyệt qua từng dòng trong cột clean_text
for text in train_df["clean_text"]:
    counter.update(text.split())

# Tổng số từ khác nhau
total_vocab = len(counter)

# Top 10 từ phổ biến nhất 
most_common = counter.most_common(10)

print(f"Tổng số từ vựng (unique words): {total_vocab:,}")
print("10 từ phổ biến nhất:")
for word, freq in most_common:
    print(f"{word:15} : {freq}")

MAX_VOCAB = 6000
vocab = {w: i+2 for i, (w, _) in enumerate(counter.most_common(MAX_VOCAB))}
vocab["<PAD>"] = 0
vocab["<UNK>"] = 1

print(f"Vocab size: {len(vocab)}")

with open("../airflow/models/vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab, f, ensure_ascii=False, indent=2)

Tổng số từ vựng (unique words): 6,668
10 từ phổ biến nhất:
giày            : 4752
đẹp             : 4153
hàng            : 3737
mua             : 2684
giao            : 2492
shop            : 2416
nên             : 2296
giá             : 1992
nhanh           : 1973
đi              : 1867
Vocab size: 6002


In [9]:
import torch

def encode(text):
    return torch.tensor([vocab.get(t, 1) for t in text.split()], dtype=torch.long)

train_df["encoded"] = train_df["clean_text"].apply(encode)
val_df["encoded"] = val_df["clean_text"].apply(encode)
test_df["encoded"] = test_df["clean_text"].apply(encode)
print(train_df["encoded"].head())
print(val_df["encoded"].head())

0    [tensor(2), tensor(3), tensor(11), tensor(57),...
1    [tensor(21), tensor(296), tensor(281), tensor(...
2    [tensor(31), tensor(44), tensor(34), tensor(18...
3    [tensor(21), tensor(76), tensor(22), tensor(25...
4    [tensor(8), tensor(5), tensor(12), tensor(58),...
Name: encoded, dtype: object
0    [tensor(4), tensor(3), tensor(119), tensor(20)...
1    [tensor(564), tensor(427), tensor(300), tensor...
2    [tensor(6), tensor(10), tensor(2), tensor(3), ...
3    [tensor(183), tensor(1794), tensor(200), tenso...
4    [tensor(194), tensor(90), tensor(12), tensor(1...
Name: encoded, dtype: object


In [10]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class ReviewDataset(Dataset):
    def __init__(self, df):
        self.texts = df["encoded"].tolist()
        self.labels = df[ASPECT_COLUMNS].values

    def __getitem__(self, idx):
        # Lấy nhãn gốc (ví dụ: [-1, 0, 1, 2])
        original_labels = torch.tensor(self.labels[idx], dtype=torch.long)
        
        # Ánh xạ nhãn: cộng 1 vào tất cả
        # -1 (None)   -> 0
        #  0 (Neg)    -> 1
        #  1 (Pos)    -> 2
        #  2 (Neu)    -> 3
        mapped_labels = original_labels + 1
        
        return self.texts[idx], mapped_labels

    def __len__(self):
        return len(self.texts)

def collate_fn(batch):
    texts, labels = zip(*batch)
    texts_padded = pad_sequence(texts, batch_first=True, padding_value=0)
    return texts_padded, torch.stack(labels)

train_loader = DataLoader(ReviewDataset(train_df), batch_size=64, shuffle=True, collate_fn=collate_fn)
val_loader   = DataLoader(ReviewDataset(val_df), batch_size=64, collate_fn=collate_fn)
test_loader = DataLoader(ReviewDataset(test_df), batch_size=64, collate_fn=collate_fn)

In [11]:
import torch
import torch.nn as nn
from sklearn.metrics import f1_score, balanced_accuracy_score


LEARNING_RATE = 1e-4
EPOCHS = 100
EMBED_DIM = 128
NUM_FILTERS = 192 # Xấp xỉ với embed dim, lớn hơn thì mạnh hơn
PATIENCE = 20                 # số epoch không cải thiện thì dừng
EVAL_EVERY = 10               # tính F1/Balanced Acc mỗi 5 epoch
WORD_WINDOW = 5
NUM_CLASSES = 4

# MODEL DEFINITION
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_labels):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.conv = nn.Conv1d(embed_dim, NUM_FILTERS, kernel_size=WORD_WINDOW, padding=int(WORD_WINDOW // 2))
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.shared_dense = nn.Linear(NUM_FILTERS, 128)
        self.dropout = nn.Dropout(0.5)
        #self.fc = nn.Linear(NUM_FILTERS, num_labels * NUM_CLASSES) # kiến trúc cũ, 8 aspects dùng chung 

        self.output_heads = nn.ModuleList([ # kiến trúc mới, mỗi aspects riêng
            nn.Linear(128, NUM_CLASSES) for _ in range(num_labels)
        ])

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1)
        x = torch.relu(self.conv(x))
        x = self.pool(x).squeeze(2)

        x = torch.relu(self.shared_dense(x)) # Cho qua lớp Dense(128)
        x = self.dropout(x)


        # return self.fc(x).view(-1, len(ASPECT_COLUMNS), NUM_CLASSES) # kiến trúc cũ
        
        # ở dưới là kiến trúc mới
        outputs = []
        for head in self.output_heads:
            outputs.append(head(x))
            
        # Xếp chồng các output lại
        # -> Shape: [batch, 8, 4]
        return torch.stack(outputs, dim=1)


# INIT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextCNN(vocab_size=len(vocab), embed_dim=EMBED_DIM, num_labels=len(ASPECT_COLUMNS)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

# TRAINING LOOP
best_val_loss = float('inf')
no_improve = 0

for epoch in range(1, EPOCHS + 1):
    # --- TRAIN ---
    model.train()
    total_loss = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)

        preds_flat = outputs.view(-1, NUM_CLASSES) 
        labels_flat = y_batch.view(-1)
        loss = criterion(preds_flat, labels_flat)

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)

    # --- VALIDATION ---
    model.eval()
    val_loss = 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)

            preds_flat = outputs.view(-1, NUM_CLASSES)
            labels_flat = y_batch.view(-1)

            loss = criterion(preds_flat, labels_flat)
            val_loss += loss.item()

            preds = outputs.argmax(dim=2).cpu().numpy()
            labels = y_batch.cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)

    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch}/{EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # --- EARLY STOPPING ---
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        no_improve = 0
        torch.save(model.state_dict(), "../airflow/models/cnn_best.pth")
    else:
        no_improve += 1
        if no_improve >= PATIENCE:
            print(f"⏹ Early stopping at epoch {epoch}")
            break

    # --- METRICS ---
    if epoch % EVAL_EVERY == 0:
        # Flatten 8 label cột để tính chung
        y_true = np.array(all_labels).flatten()
        y_pred = np.array(all_preds).flatten()

        all_known_labels = [0, 1, 2, 3] # 0=None, 1=Neg, 2=Pos, 3=Neu
        
        # Tính F1 trên cả 4 lớp
        macro_f1 = f1_score(y_true, y_pred, average="macro", labels=all_known_labels, zero_division=0)
        balanced_acc = balanced_accuracy_score(y_true, y_pred)

        print(f"\nEpoch {epoch}: Macro F1 = {macro_f1:.4f} | Balanced Acc = {balanced_acc:.4f}\n")

print("Training complete. Best model saved to cnn_best.pth")


/media/thsondev/SSD/Code/appbigdata/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/thsondev/SSD/Code/appbigdata/.venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
/media/thsondev/SSD/Code/appbigdata/.venv/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally 

Epoch 1/100 | Train Loss: 0.9069 | Val Loss: 0.6845
Epoch 2/100 | Train Loss: 0.6977 | Val Loss: 0.6107
Epoch 3/100 | Train Loss: 0.6219 | Val Loss: 0.5546
Epoch 4/100 | Train Loss: 0.5719 | Val Loss: 0.5125
Epoch 5/100 | Train Loss: 0.5333 | Val Loss: 0.4792
Epoch 6/100 | Train Loss: 0.4971 | Val Loss: 0.4533
Epoch 7/100 | Train Loss: 0.4703 | Val Loss: 0.4325
Epoch 8/100 | Train Loss: 0.4470 | Val Loss: 0.4162
Epoch 9/100 | Train Loss: 0.4305 | Val Loss: 0.4043
Epoch 10/100 | Train Loss: 0.4118 | Val Loss: 0.3906

Epoch 10: Macro F1 = 0.5042 | Balanced Acc = 0.4701

Epoch 11/100 | Train Loss: 0.3970 | Val Loss: 0.3794


/media/thsondev/SSD/Code/appbigdata/.venv/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch 12/100 | Train Loss: 0.3835 | Val Loss: 0.3719
Epoch 13/100 | Train Loss: 0.3704 | Val Loss: 0.3623
Epoch 14/100 | Train Loss: 0.3620 | Val Loss: 0.3567
Epoch 15/100 | Train Loss: 0.3487 | Val Loss: 0.3516
Epoch 16/100 | Train Loss: 0.3388 | Val Loss: 0.3462
Epoch 17/100 | Train Loss: 0.3301 | Val Loss: 0.3407
Epoch 18/100 | Train Loss: 0.3202 | Val Loss: 0.3368
Epoch 19/100 | Train Loss: 0.3110 | Val Loss: 0.3348
Epoch 20/100 | Train Loss: 0.3047 | Val Loss: 0.3298

Epoch 20: Macro F1 = 0.6352 | Balanced Acc = 0.5747

Epoch 21/100 | Train Loss: 0.2973 | Val Loss: 0.3292
Epoch 22/100 | Train Loss: 0.2885 | Val Loss: 0.3253


/media/thsondev/SSD/Code/appbigdata/.venv/lib/python3.12/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


...

Epoch 24/100 | Train Loss: 0.2734 | Val Loss: 0.3206
Epoch 25/100 | Train Loss: 0.2682 | Val Loss: 0.3202
Epoch 26/100 | Train Loss: 0.2605 | Val Loss: 0.3182
Epoch 27/100 | Train Loss: 0.2555 | Val Loss: 0.3165
Epoch 28/100 | Train Loss: 0.2504 | Val Loss: 0.3173
Epoch 29/100 | Train Loss: 0.2436 | Val Loss: 0.3174
Epoch 30/100 | Train Loss: 0.2368 | Val Loss: 0.3189

Epoch 30: Macro F1 = 0.6753 | Balanced Acc = 0.6152

Epoch 31/100 | Train Loss: 0.2325 | Val Loss: 0.3155
Epoch 32/100 | Train Loss: 0.2259 | Val Loss: 0.3155
Epoch 33/100 | Train Loss: 0.2221 | Val Loss: 0.3172


/media/thsondev/SSD/Code/appbigdata/.venv/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch 34/100 | Train Loss: 0.2148 | Val Loss: 0.3165
Epoch 35/100 | Train Loss: 0.2114 | Val Loss: 0.3182
Epoch 36/100 | Train Loss: 0.2070 | Val Loss: 0.3192
Epoch 37/100 | Train Loss: 0.2007 | Val Loss: 0.3198
Epoch 38/100 | Train Loss: 0.1975 | Val Loss: 0.3209
Epoch 39/100 | Train Loss: 0.1941 | Val Loss: 0.3213
Epoch 40/100 | Train Loss: 0.1893 | Val Loss: 0.3207

Epoch 40: Macro F1 = 0.6954 | Balanced Acc = 0.6423

Epoch 41/100 | Train Loss: 0.1842 | Val Loss: 0.3229


/media/thsondev/SSD/Code/appbigdata/.venv/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch 42/100 | Train Loss: 0.1788 | Val Loss: 0.3274
Epoch 43/100 | Train Loss: 0.1767 | Val Loss: 0.3270
Epoch 44/100 | Train Loss: 0.1761 | Val Loss: 0.3308
Epoch 45/100 | Train Loss: 0.1690 | Val Loss: 0.3285
Epoch 46/100 | Train Loss: 0.1663 | Val Loss: 0.3311
Epoch 47/100 | Train Loss: 0.1606 | Val Loss: 0.3315
Epoch 48/100 | Train Loss: 0.1587 | Val Loss: 0.3327
Epoch 49/100 | Train Loss: 0.1546 | Val Loss: 0.3398
Epoch 50/100 | Train Loss: 0.1511 | Val Loss: 0.3372

Epoch 50: Macro F1 = 0.7010 | Balanced Acc = 0.6520

Epoch 51/100 | Train Loss: 0.1489 | Val Loss: 0.3392
⏹ Early stopping at epoch 51
Training complete. Best model saved to cnn_best.pth


In [12]:
print("Đang tải mô hình 'cnn_best.pth'...")
model = TextCNN(vocab_size=len(vocab), embed_dim=EMBED_DIM, num_labels=len(ASPECT_COLUMNS)).to(device)
model.load_state_dict(torch.load("../airflow/models/cnn_best.pth", map_location=device))

model.eval()

print("Đang dự đoán trên tập test")
all_preds = []
all_labels = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        outputs = model(X_batch)
        preds = outputs.argmax(dim=2) # Shape: (batch_size, 8)
        
        all_preds.append(preds)
        all_labels.append(y_batch)
        
# Nối các batch lại thành tensor lớn
all_preds_tensor = torch.cat(all_preds, dim=0)
all_labels_tensor = torch.cat(all_labels, dim=0)

# Chuyển về numpy
y_true_all = all_labels_tensor.cpu().numpy() # Shape: (N_samples, 8)
y_pred_all = all_preds_tensor.cpu().numpy() # Shape: (N_samples, 8)

print("Đã hoàn tất dự đoán. Bắt đầu tính toán metrics...")

# Làm phẳng tất cả các aspect để tính 1 chỉ số tổng quan
y_true_flat = y_true_all.flatten()
y_pred_flat = y_pred_all.flatten()

all_known_labels = [0, 1, 2, 3] # 0=None, 1=Neg, 2=Pos, 3=Neu

# [PHẦN TÍNH OVERALL METRICS]
# BỎ LỌC (MASK)
# y_true_flat và y_pred_flat giờ chứa các nhãn [0, 1, 2, 3]

overall_f1 = f1_score(y_true_flat, y_pred_flat, average="macro", labels=all_known_labels, zero_division=0)
overall_acc = balanced_accuracy_score(y_true_flat, y_pred_flat)

print("\n--- KẾT QUẢ TỔNG QUAN TRÊN TẬP TEST ---")
print(f"Overall Macro F1-Score:    {overall_f1:.4f}")
print(f"Overall Balanced Accuracy: {overall_acc:.4f}")

# --- 6. Tính toán Metrics CHO TỪNG ASPECT ---
aspect_metrics = {}

for i, aspect in enumerate(ASPECT_COLUMNS):
    y_true_aspect = y_true_all[:, i]
    y_pred_aspect = y_pred_all[:, i]
    
    
    if len(y_true_aspect) > 0:
        f1 = f1_score(y_true_aspect, y_pred_aspect, average="macro", zero_division=0)
        acc = balanced_accuracy_score(y_true_aspect, y_pred_aspect)
        aspect_metrics[aspect] = {"f1": f1, "acc": acc}
    else:
        aspect_metrics[aspect] = {"f1": 0.0, "acc": 0.0} # Không có nhãn nào để đánh giá

# --- 7. Tìm Aspect Tốt/Tệ Nhất (dựa trên F1-Score) ---
# Sắp xếp dict theo F1-score
sorted_aspects = sorted(aspect_metrics.items(), key=lambda item: item[1]['f1'])

print("\nBảng chi tiết (sắp xếp theo F1):")
for aspect, metrics in sorted_aspects:
    print(f"  - {aspect:15}: F1 = {metrics['f1']:.4f} | Bal. Acc = {metrics['acc']:.4f}")

Đang tải mô hình 'cnn_best.pth'...
Đang dự đoán trên tập test
Đã hoàn tất dự đoán. Bắt đầu tính toán metrics...

--- KẾT QUẢ TỔNG QUAN TRÊN TẬP TEST ---
Overall Macro F1-Score:    0.6674
Overall Balanced Accuracy: 0.6100

Bảng chi tiết (sắp xếp theo F1):
  - General        : F1 = 0.4523 | Bal. Acc = 0.4097
  - Outlook        : F1 = 0.5078 | Bal. Acc = 0.4947
  - Price          : F1 = 0.5208 | Bal. Acc = 0.4865
  - Shop_Service   : F1 = 0.5315 | Bal. Acc = 0.5041
  - Quality        : F1 = 0.5394 | Bal. Acc = 0.5050
  - Size           : F1 = 0.6107 | Bal. Acc = 0.5872
  - Shipping       : F1 = 0.6743 | Bal. Acc = 0.6665
  - Others         : F1 = 0.8402 | Bal. Acc = 0.7921


/media/thsondev/SSD/Code/appbigdata/.venv/lib/python3.12/site-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


In [13]:
custom_reviews = [
    "Giày đẹp lắm shop, giao hàng cũng nhanh nữa, giá rẻ quá.",
    "Rất tệ, giày bị hỏng, có mùi hôi và shop không trả lời tin nhắn.",
    "Đóng gói bình thường, chất lượng cũng tạm ổn so với giá tiền.",
    "Size hơi nhỏ so với mô tả, tôi phải đi đổi lại."
]

prediction_map = {
    0: "NONE", # Lớp 0 (trước là -1)
    1: "NEG",   # Lớp 1 (trước là 0)
    2: "POS",   # Lớp 2 (trước là 1)
    3: "NEU"    # Lớp 3 (trước là 2)
}

model = TextCNN(vocab_size=len(vocab), embed_dim=EMBED_DIM, num_labels=len(ASPECT_COLUMNS)).to(device)
model.load_state_dict(torch.load("../airflow/models/cnn_best.pth", map_location=device))
model.eval() 

cleaned_texts = [clean(text) for text in custom_reviews]
encoded_texts = [encode(text) for text in cleaned_texts]
padded_batch = pad_sequence(encoded_texts, batch_first=True, padding_value=0).to(device)

with torch.no_grad():
    outputs = model(padded_batch)
    predictions = outputs.argmax(dim=2).cpu().numpy()

# In kết quả

for i, review in enumerate(custom_reviews):
    print("\n---------------------------------")
    print(f"REVIEW: \"{review}\"")
    
    review_preds = predictions[i] # Lấy mảng 8 dự đoán cho review này
    
    for j, aspect in enumerate(ASPECT_COLUMNS):
        pred_index = review_preds[j] # Lấy dự đoán (0, 1, 2)
        pred_string = prediction_map.get(pred_index, "LỖI") # Map sang text
        
        print(f"  - {aspect:15}: {pred_string}")
        


---------------------------------
REVIEW: "Giày đẹp lắm shop, giao hàng cũng nhanh nữa, giá rẻ quá."
  - Price          : POS
  - Shipping       : POS
  - Outlook        : POS
  - Quality        : NONE
  - Size           : NONE
  - Shop_Service   : NONE
  - General        : NONE
  - Others         : NONE

---------------------------------
REVIEW: "Rất tệ, giày bị hỏng, có mùi hôi và shop không trả lời tin nhắn."
  - Price          : NONE
  - Shipping       : NONE
  - Outlook        : NONE
  - Quality        : NONE
  - Size           : NONE
  - Shop_Service   : NEG
  - General        : NONE
  - Others         : NONE

---------------------------------
REVIEW: "Đóng gói bình thường, chất lượng cũng tạm ổn so với giá tiền."
  - Price          : NONE
  - Shipping       : NONE
  - Outlook        : NONE
  - Quality        : NEU
  - Size           : NONE
  - Shop_Service   : NEG
  - General        : NEU
  - Others         : NONE

---------------------------------
REVIEW: "Size hơi nhỏ so với 